In [1]:
#preprocess
import json
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# data process

In [2]:
df = pd.read_csv('../data/all.csv')

In [3]:
df.head()

,Unnamed: 0,user_no,key_label,event,event_time,post_id,news_time,title,abstract,content,industry_name_sc,concept_name_sc,label_tag,input,label,sentence1_key
0,0,929,0,newsView,2022-07-28 23:56:17,13011492,2022-07-28 23:28:38,Market Movers | Solar stocks surge Thursday af...,Solar stocks were surging on Thursday morning ...,"<div class=""ftEditor""><p>11:02 AM EDT, 07/28/2...","[""太阳能"",""电气设备及零件""]","[""法国巴黎银行持仓"",""昨日强势股"",""光伏太阳能"",""氢能源"",""荷兰全球保险持仓"",""...","""太阳能"",""电气设备及零件""",title: Market Movers | Solar stocks surge Thur...,"industry: [""太阳能"",""电气设备及零件""]",title: Market Movers | Solar stocks surge Thur...
1,1,1659,0,newsView,2022-07-28 12:03:26,12786302,2022-07-22 18:56:21,COSCO Shipping Parent Lifts Stake in Firm to O...,"06:54 AM EDT, 07/22/2022 (MT Newswires) -- COS...","<p>06:54 AM EDT, 07/22/2022 (MT Newswires) -- ...","[""航运及港口"",""航运港口""]","[""一带一路"",""港口运输股"",""摘帽"",""转融券标的"",""国企改革"",""BDI指数"",""证...","""航运及港口"",""航运港口""",title: COSCO Shipping Parent Lifts Stake in Fi...,"industry: [""航运及港口"",""航运港口""]",title: COSCO Shipping Parent Lifts Stake in Fi...
2,2,1659,0,newsView,2022-07-28 12:03:59,12622443,2022-07-19 07:40:55,Does COSCO SHIPPING Holdings (HKG:1919) Have A...,The external fund manager backed by Berkshire ...,<p>The external fund manager backed by Berkshi...,"[""航运及港口""]","[""一带一路"",""港口运输股""]","""航运及港口""",title: Does COSCO SHIPPING Holdings (HKG:1919)...,"industry: [""航运及港口""]",title: Does COSCO SHIPPING Holdings (HKG:1919)...
3,3,1659,0,newsView,2022-07-28 12:03:59,12622443,2022-07-19 07:40:55,Does COSCO SHIPPING Holdings (HKG:1919) Have A...,The external fund manager backed by Berkshire ...,The external fund manager backed by Berkshire ...,"[""航运港口""]","[""一带一路"",""摘帽"",""转融券标的"",""国企改革"",""BDI指数"",""证金持股"",""上证...","""航运港口""",title: Does COSCO SHIPPING Holdings (HKG:1919)...,"industry: [""航运港口""]",title: Does COSCO SHIPPING Holdings (HKG:1919)...
4,4,1659,0,newsView,2022-07-28 12:06:26,12252154,2022-07-07 15:22:00,Cosco Shipping Expects 74% Rise in H1 Net Prof...,"03:20 AM EDT, 07/07/2022 (MT Newswires) -- Cos...","<p>03:20 AM EDT, 07/07/2022 (MT Newswires) -- ...","[""航运及港口""]","[""一带一路"",""港口运输股""]","""航运及港口""",title: Cosco Shipping Expects 74% Rise in H1 N...,"industry: [""航运及港口""]",title: Cosco Shipping Expects 74% Rise in H1 N...


In [4]:
#fillna
df = df[-df['sentence1_key'].isnull()]
df = df[-df['label'].isnull()]

In [5]:
train, test = train_test_split(df,test_size=0.25,random_state=0)
print ("train size {}, test size{}".format(train.shape,test.shape))

train size (14035, 16), test size(4679, 16)


In [6]:
train[['sentence1_key','label']].head()

,sentence1_key,label
9354,title: Facebook Stops Renewing US Publisher Co...,"industry: [""互联网内容与信息""]"
7558,title: Xiaomi Announces to Push back MIUI 13 S...,"industry: [""电讯设备""]"
10860,title: Why XPeng shares soared over 10% today?...,"industry: [""汽车制造商"",""汽车""]"
6687,title: Bilibili Stock Is 8% Down At Session St...,"industry: [""电子游戏与多媒体""]"
4730,"title: 为破解缺芯困局 车企集体开启汽车芯片“定制”模式 ,abstract: 过去两...","industry: [""半导体"",""汽车制造商""]"


In [7]:
import os

path = '../data/train'

# Check whether the specified path exists or not
isExist = os.path.exists(path)

if not isExist:
    os.makedirs(path)
    print("The new directory is created!")
    
train[["label","sentence1_key"]].to_csv('../data/train/train.csv',index=False,encoding='utf-8')
test[["label","sentence1_key"]].to_csv('../data/train/test.csv',index=False,encoding='utf-8')

# train - model b

In [8]:
import boto3
import sagemaker
import os
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()

prefix='futu-datalab'

bucket = sess.default_bucket() 
boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "train/train.csv")
).upload_file("../data/train/train.csv")
boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "test/test.csv")
).upload_file("../data/train/test.csv")

training_input_path = f's3://{sess.default_bucket()}/{prefix}/train/train.csv'
test_input_path = f's3://{sess.default_bucket()}/{prefix}/test/test.csv'

In [ ]:
from sagemaker.huggingface import HuggingFace
from sagemaker.huggingface import TrainingCompilerConfig

#speed up use sagemaker compiler https://towardsdatascience.com/speed-up-hugging-face-training-jobs-on-aws-by-up-to-50-with-sagemaker-training-compiler-9ad2ac5b0eb

# hyperparameters which are passed to the training job
hyperparameters={'reference_column':'sentence1_key',
                 'hypothesis_column':'label',
                 'train_file':'/opt/ml/input/data/train/train.csv',
                 'validation_file':'/opt/ml/input/data/validation/test.csv',
                 'test_file':'/opt/ml/input/data/test/test.csv',
                 'output_dir':'/opt/ml/model',
                 'do_train':True,
                 'do_eval':True,
                 'max_source_length': 128,
                 'max_target_length': 128,
                 'model_name_or_path': 'google/mt5-small',
                 'learning_rate': 3e-4,
                 'num_train_epochs': 10,
                 'per_device_train_batch_size': 2,#16
                 'gradient_accumulation_steps':2, 
                 'save_strategy':'steps',
                 'evaluation_strategy':'epoch',
                 'save_total_limit':1,
                 'eval_steps':5000,
                 'predict_with_generate':True # customerized accuracy
                 }

# create the Estimator
huggingface_estimator = HuggingFace(
        entry_point='run_train.py',
        source_dir='./scripts',
        instance_type='ml.p3.2xlarge',#'ml.p3dn.24xlarge'
        instance_count=1,
        role=role,
        max_run=24*60*60,
        transformers_version='4.6',
        pytorch_version='1.7',
        py_version='py36',
        volume_size=128,
        #compiler_config=TrainingCompilerConfig(),
        base_job_name='futu-stary-10epoch',
        hyperparameters = hyperparameters,
#         distribution=distribution
)

huggingface_estimator.fit({'train':training_input_path,'test':test_input_path,'validation': test_input_path})

2022-09-14 12:07:29 Starting - Starting the training job...
2022-09-14 12:07:56 Starting - Preparing the instances for trainingProfilerReport-1663157249: InProgress
.........
2022-09-14 12:09:16 Downloading - Downloading input data...
2022-09-14 12:09:53 Training - Downloading the training image.......

# deploy

In [1]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker

role = sagemaker.get_execution_role()

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data="s3://sagemaker-us-east-1-726335585155/train-modelb-stary-0713-crossbook-2022-07-14-06-27-10-862/output/model.tar.gz",  # path to your trained sagemaker model
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.6", # transformers version used
   pytorch_version="1.7", # pytorch version used
   py_version="py36", # python version of the DLC
)


In [2]:
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.g4dn.xlarge"
)

-------!

In [3]:
%%time
# example request, you always need to define "inputs"
import time


data = {
   "inputs": 'prefix: "Whatever. Chris has a heir to the pack now." She says.,"Um last time I remembered in the laws it was clearly written that the offspring of the  Alpha can only take over if he is the son of the Alpha and the Luna and clearly after today no one will take this mistake," I point to her stomach , center: "seriously again as you are just a mistress not the mate." I say putting a lot of emphasis on the word \'mistress\' as if it is the world\'s most disgusting word. after: "seriously again as you are just a mistress not the mate." I say putting a lot of emphasis on the word \'mistress\' as if it is the world\'s most disgusting word.,"Well at least he loves me." She says desperately. roles: [\'Skylar\', \'Logan\']'
}

# request
predictor.predict(data)


CPU times: user 13.3 ms, sys: 0 ns, total: 13.3 ms
Wall time: 864 ms


[{'generated_text': 'Annie said the sentence'}]

In [4]:
import pandas as pd
df = pd.read_csv('../model_b_data/test.csv')

In [5]:
res = []
for i in df['sentence1_key']:
    data = {"inputs": i}

    # request
    res.append(predictor.predict(data))

In [6]:
#p f r
from sklearn.metrics import classification_report

y_pred = [i[0]['generated_text'] for i in res]
y_true = df['label']
 
print(classification_report(y_true, y_pred))

                         precision    recall  f1-score   support

Annie said the sentence       0.96      0.99      0.97       211
  Ava said the sentence       0.96      0.99      0.97       210
 Kade said the sentence       0.97      0.93      0.95       120
 Zach said the sentence       0.98      0.95      0.96       146

               accuracy                           0.97       687
              macro avg       0.97      0.96      0.96       687
           weighted avg       0.97      0.97      0.97       687



In [60]:
df['sentence1_key'][0]

'prefix: "Whatever. Chris has a heir to the pack now." She says.,"Um last time I remembered in the laws it was clearly written that the offspring of the  Alpha can only take over if he is the son of the Alpha and the Luna and clearly after today no one will take this mistake," I point to her stomach , center: "seriously again as you are just a mistress not the mate." I say putting a lot of emphasis on the word \'mistress\' as if it is the world\'s most disgusting word. after: "seriously again as you are just a mistress not the mate." I say putting a lot of emphasis on the word \'mistress\' as if it is the world\'s most disgusting word.,"Well at least he loves me." She says desperately. roles: [\'Skylar\', \'Logan\']'

In [61]:
df['label'][0]

'Logan said the sentence'

In [57]:
x = 0
for i in range(len(y_true)):
    if y_true[i]==y_pred[i]:
        x = x+1

In [59]:
x/len(y_true)

0.838

In [23]:
df['sentence1_key'][362]

'"Who do you think I am? Of course I want it." She whispers and look at her stomach. '

In [24]:
df['sentence1_key'][370]

'"What do you mean by that?" She asks me surprised. '